Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [365]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
from heapq import heappush, heappop, heapify
import functools

In [366]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [367]:
class State:
    def __init__(self, content: np.ndarray):
        self.content: np.ndarray = content
        self.hash: int = hash(content.tobytes())
        
    def __hash__(self):
        return self.hash

    def __eq__(self, other):
        return self.hash == other.hash
    
    def __lt__(self, other):
        return self.hash < other.hash

In [368]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(i[0]) for i in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def counter(fn):

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper

@counter
def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

## A* algorithm 

In [ ]:
def get_pos(state_content: np.ndarray, n: int) -> tuple[int, int]:
    x, y = np.argwhere(state_content== n)[0]
    return x, y

def manhattan_distance(state: np.ndarray, goal: np.ndarray, n: int) -> int:
	x1, y1 = get_pos(state, n)
	x2, y2 = get_pos(goal, n)
	return abs(x1 - x2) + abs(y1 - y2)

def heuristic(state: np.ndarray, goal: np.ndarray) -> int:
    return sum([manhattan_distance(state, goal, n) for n in range(1, PUZZLE_DIM**2)]) // 2

def quality(solution: list[int]) -> int:
    return len(solution)

def astar(state: np.ndarray, goal: np.ndarray) -> float:
    
	state, goal = map(State, [state, goal])
	open_set = []
	closed_set = set()
	parents = {}
	past_len = {state.hash: 0}
 
	heappush(open_set, (0, state, []))
	closed_set.add(state.hash)

	current = state
	while open_set:
		_, current, curr_path = heappop(open_set)

		if current.hash == goal.hash:
			return True, curr_path

		for action in available_actions(current.content):
			neighbor = State(do_action(current.content, action))

			if neighbor.hash not in closed_set or past_len[current.hash] + 1 < past_len[neighbor.hash]:
				parents[neighbor.hash] = current.hash
				past_len[neighbor.hash] = quality(curr_path) + 1
				cost = past_len[neighbor.hash] + heuristic(neighbor.content, goal.content)
				heappush(open_set, (cost, neighbor, curr_path + [action]))
				closed_set.add(neighbor.hash)
    
	return False, []

In [370]:
solver = astar

In [371]:
content = np.random.choice(range(PUZZLE_DIM**2), (PUZZLE_DIM, PUZZLE_DIM), replace=False)
goal = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

print(content)

success, path = solver(content, goal)
quality, cost = quality(path), do_action.calls
print(success, quality, cost)
path

[[7 8 3]
 [4 6 0]
 [1 2 5]]
False 0 483840


[]